In [94]:
from Patient import patient
from Record import record

In [95]:
import numpy as np
import pandas as pd

In [96]:
#下面有範例

In [101]:
class main():
    def __init__(self):
        #資料庫
        self.patients_data = np.nan
        self.records_data = np.nan
        self.inpatients_data = np.nan
        #選擇的人
        self.main_patient = np.nan
        self.main_record = np.nan
        
    #load 資料庫
    def load_data(self):
        self.patients_data = pd.read_pickle('patients_100.pickle')
        self.records_data = pd.read_pickle('records_100.pickle')
        self.inpatients_data = pd.read_pickle('inpatients_100.pickle')
        
    #用ID load 人    
    def set_patient_by_id(self, _id):
        patients = self.patients_data
        records = self.records_data
        inpatients = self.inpatients_data
        patients = patients[patients['ID']==_id]
        inpatients = inpatients.drop('HospID',axis = 1)
        inpatients['Type'] = 2.0
        records['Type'] = records['Type'].astype('int64')
        records = records[records['Type']!=2]
        records = pd.concat([records,inpatients])
        records = records.reset_index(drop='True')
        records = records[records['ID']==_id]

        self.main_patient = patient(patients.ID.values[0],
                               patients.Gender.values[0],
                               patients.Birth.values[0],
                               patients.Death.values[0])
        self.main_record = list()
        i=0
        for row in records.itertuples():
            self.main_record.append(record(i,
                               getattr(row,'InDate'),
                               getattr(row,'Type'),
                               getattr(row,'Area'),
                               getattr(row,'TotalDot'),
                               getattr(row,'ICD9CM'),
                               getattr(row,'Drug')))
            i=i+1
    
    def get_patient_personal_data(self):
        out = pd.DataFrame([[
            self.main_patient._id,
            self.main_patient._gender,
            np.datetime_as_string(self.main_patient._birthdate, unit='D')
        ]],columns=['id','gender','birthdate'])
        return out
    
    def get_patient_records(self):
        out_dataframe = list()
        for record in self.main_record:
            out = list()
            out.append(record._id)
            out.append(record._date)
            out.append(record._type)
            out.append(record._diseases[0])
            out_dataframe.append(out)
        return pd.DataFrame(out_dataframe,columns=['id','date','type','icd'])
    
    def get_record_detial(self, _id):
        i=0
        while(1):
            if(self.main_record[i]._id == _id):
                out = pd.DataFrame([[
                    self.main_patient._id,
                    self.main_patient._gender,
                    self.main_record[i]._id,
                    self.main_record[i]._type,
                    self.main_record[i]._date,
                    self.main_record[i]._diseases,
                    self.main_record[i]._medicine
                ]],columns=['id','gender','record_id','type','date','icd','drug'])
                break
            else:
                i = i+1
        return out

In [102]:
test = main()

In [103]:
%%time
test.load_data()

Wall time: 19.9 ms


In [104]:
%%time
test_patient_ID = '0007613eee440aa3145f09e1dfd0865b'
test.set_patient_by_id(test_patient_ID)

Wall time: 42.9 ms


In [105]:
%%time
test.get_patient_personal_data()

Wall time: 989 µs


,id,gender,birthdate
0,0007613eee440aa3145f09e1dfd0865b,M,1968-07-01


In [106]:
%%time
test.get_patient_records()

Wall time: 1.99 ms


,id,date,type,icd
0,0,1998-09-07,1.0,410
1,1,1999-01-14,1.0,682
2,2,1999-01-25,0.0,682
3,3,1999-01-27,0.0,682
4,4,1999-01-28,0.0,682
...,...,...,...,...
445,445,2000-07-04,2.0,44422
446,446,2000-08-08,2.0,7854
447,447,2010-12-13,2.0,389
448,448,2011-11-05,2.0,51881


In [107]:
%%time
#找地0筆record
_id = 0 
test.get_record_detial(_id)

Wall time: 997 µs


,id,gender,record_id,type,date,icd,drug
0,0007613eee440aa3145f09e1dfd0865b,M,0,1.0,1998-09-07,"[410, 410, 413]",NaN
